In [ ]:
import fastf1
import pandas as pd
from datetime import datetime
import os
import logging
import sys

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler("f1_weather_data.log")
    ]
)

# Define cache and output directories
CACHE_DIR = 'cache'  # Ensure this is the correct path where FastF1 caches data
OUTPUT_DIR = 'f1_weather_data'

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Enable caching with specified directory
fastf1.Cache.enable_cache(CACHE_DIR)

def get_session_weather(session):
    """
    Extract weather data from a session and return as a DataFrame.
    """
    try:
        weather = session.weather
        # Create a DataFrame from weather data
        weather_data = pd.DataFrame([{
            'Temperature': weather.temp,
            'Humidity': weather.humidity,
            'Pressure': weather.pressure,
            'WindSpeed': weather.wind_speed,
            'WindDirection': weather.wind_direction,
            'Rain': weather.rain,
            'Year': session.event.year,
            'GrandPrix': session.event.name,
            'Session': session.name,
            'SessionType': type(session).__name__,
            'Date': session.date
        }])
        return weather_data
    except Exception as e:
        logging.error(f"Error getting weather data for {session.event.name} - {session.name}: {str(e)}")
        return None

def get_race_weekend_weather(year, gp):
    """
    Get weather data for all sessions in a race weekend.
    """
    all_weather_data = []
    
    # List of possible session types
    sessions = ['FP1', 'FP2', 'FP3', 'Q', 'S', 'SS', 'R']
    
    for session_name in sessions:
        try:
            session = fastf1.get_session(year, gp, session_name)
            session.load()
            weather_data = get_session_weather(session)
            if weather_data is not None:
                all_weather_data.append(weather_data)
        except Exception as e:
            logging.warning(f"Could not load {session_name} for {year} {gp}: {str(e)}")
            continue
    
    if all_weather_data:
        return pd.concat(all_weather_data, ignore_index=True)
    return None

def main():
    # Get current year
    current_year = datetime.now().year
    
    # Define the range of years to process
    start_year = 2018  # FastF1 data starts from 2018
    years = list(range(start_year, current_year + 1))
    
    # Initialize list to collect all data
    all_data = []
    
    for year in years:
        try:
            # Get schedule for the year
            schedule = fastf1.get_event_schedule(year)
            if schedule.empty:
                logging.warning(f"No events found for year {year}. Skipping.")
                continue
            
            for _, race in schedule.iterrows():
                logging.info(f"Processing {year} {race['EventName']}")
                weather_data = get_race_weekend_weather(year, race['EventName'])
                
                if weather_data is not None:
                    all_data.append(weather_data)
        
        except Exception as e:
            logging.error(f"Error processing year {year}: {str(e)}")
            continue
    
    if all_data:
        # Combine all data into a single DataFrame
        final_df = pd.concat(all_data, ignore_index=True)
        
        # Ensure 'Date' is datetime
        final_df['Date'] = pd.to_datetime(final_df['Date'])
        
        # Sort by date and time
        final_df.sort_values(['Year', 'GrandPrix', 'Session', 'Date'], inplace=True)
        
        # Save to a single CSV file
        output_file = os.path.join(OUTPUT_DIR, 'f1_weather_data_all.csv')
        final_df.to_csv(output_file, index=False)
        logging.info(f"\nWeather data successfully saved to {output_file}")
        
        # Print basic statistics
        logging.info(f"\nData collected:")
        logging.info(f"Years covered: {final_df['Year'].unique()}")
        logging.info(f"Number of Grand Prix: {len(final_df['GrandPrix'].unique())}")
        logging.info(f"Total weather records: {len(final_df)}")
    else:
        logging.warning("No data was collected.")

if __name__ == "__main__":
    main()


In [1]:
import fastf1
import os
import logging
import sys
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# Configure logging
logging.basicConfig(
    level=logging.WARNING,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler("f1_weather_data.log")
    ]
)

# Enable cache
fastf1.Cache.enable_cache('cache')  # Use default cache path if already set
fastf1.Cache.offline_mode(enabled=True)

def process_year(year):
    """
    Process all events for a given year and return a dataframe of all laps.
    """
    try:
        print(f"Processing year {year}")
        lap_dataframes = []
        # Get the event schedule for the year
        schedule = fastf1.get_event_schedule(year)
        # For each event in the schedule
        for event in schedule.itertuples():
            event_name = event.EventName
            event_round = event.RoundNumber
            print(f"Processing event: {event_name}, year: {year}")
            session_names = ['FP1', 'FP2', 'FP3', 'Q', 'S', 'SS', 'R']
            # Process each session
            for session_name in session_names:
                try:
                    print(f"Processing session: {session_name} for event: {event_name}, year: {year}")
                    # Load the session
                    session = fastf1.get_session(year, event_round, session_name)
                    session.load()
                    # Get the laps data
                    laps_df = session.laps.copy()
                    # Add additional context columns
                    laps_df['Year'] = year
                    laps_df['EventName'] = event_name
                    laps_df['SessionName'] = session_name
                    lap_dataframes.append(laps_df)
                except Exception as e:
                    print(f"Could not process session {session_name} for event {event_name} in year {year}: {e}")
        # Concatenate all laps for the year
        return pd.concat(lap_dataframes, ignore_index=True) if lap_dataframes else None
    except Exception as e:
        print(f"Error processing year {year}: {e}")
        return None

# Define years to process
years = range(2018, 2024)  # From 2018 to 2023 inclusive

# Start a thread pool executor for parallel processing
lap_dataframes = []
with ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_year, year): year for year in years}
    for future in as_completed(futures):
        year = futures[future]
        try:
            year_data = future.result()
            if year_data is not None:
                lap_dataframes.append(year_data)
        except Exception as e:
            print(f"Error in processing year {year}: {e}")

# Combine all years' data into a single dataframe
if lap_dataframes:
    all_lap_data = pd.concat(lap_dataframes, ignore_index=True)
    # Save the combined dataframe to a CSV file
    all_lap_data.to_csv('ff1_laps.csv', index=False)
    print("Lap data collection complete. Data saved to 'ff1_laps.csv'.")
else:
    print("No data collected.")


logger      WARNING 	Failed to load schedule from FastF1 backend!


Processing year 2018Processing year 2019

Processing year 2020
Processing year 2021
Processing year 2022
Processing year 2023
2024-11-28 11:42:29,440 - WARNING - Failed to load schedule from FastF1 backend!


logger      WARNING 	Failed to load schedule from FastF1 backend!
logger      WARNING 	Failed to load schedule from FastF1 backend!
logger      WARNING 	Failed to load schedule from FastF1 backend!


Processing event: Australian Grand Prix, year: 2018Processing event: Pre-Season Track Session, year: 2022
Processing session: FP1 for event: Pre-Season Track Session, year: 2022

Processing session: FP1 for event: Australian Grand Prix, year: 2018
Processing event: Pre-Season Test 1, year: 2020
Processing session: FP1 for event: Pre-Season Test 1, year: 2020
Processing event: Pre-Season Test, year: 2021
Processing session: FP1 for event: Pre-Season Test, year: 2021
Could not process session FP1 for event Pre-Season Test in year 2021: Cannot get testing event by round number!
Processing session: FP2 for event: Pre-Season Test, year: 2021
Could not process session FP2 for event Pre-Season Test in year 2021: Cannot get testing event by round number!
Processing session: FP3 for event: Pre-Season Test, year: 2021
Could not process session FP3 for event Pre-Season Test in year 2021: Cannot get testing event by round number!
Processing session: Q for event: Pre-Season Test, year: 2021
Could n

core           INFO 	Loading data for Bahrain Grand Prix - Practice 1 [v3.4.3]


...

2024-11-28 11:55:48,429 - INFO - Using cached data for weather_data


req            INFO 	Using cached data for race_control_messages


2024-11-28 11:55:48,439 - INFO - Using cached data for race_control_messages


core           INFO 	Finished loading data for 20 drivers: ['33', '44', '55', '22', '10', '77', '4', '14', '31', '16', '5', '3', '18', '47', '11', '6', '99', '63', '7', '9']


2024-11-28 11:55:48,474 - INFO - Finished loading data for 20 drivers: ['33', '44', '55', '22', '10', '77', '4', '14', '31', '16', '5', '3', '18', '47', '11', '6', '99', '63', '7', '9']
Lap data collection complete. Data saved to 'ff1_laps.csv'.
